In [117]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import backtrader as bt
from datetime import datetime
import statsmodels
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller

In [195]:
soxx_data = yf.download("SOXX", start='2021-01-01')
nvda_data = yf.download("NVDA", start='2021-01-01')
tsmc_data = yf.download("TSM", start='2021-01-01')
intc_data = yf.download("INTC", start='2021-01-01')
nxpi_data = yf.download("NXPI", start='2021-01-01')
avgo_data = yf.download("AVGO", start='2021-01-01')
txn_data = yf.download("TXN", start='2021-01-01')
amd_data = yf.download("AMD", start='2021-01-01')
data_list = [soxx_data, nvda_data, tsmc_data, intc_data, nxpi_data, avgo_data, txn_data, amd_data]
data_list_names = ["soxx_data", "nvda_data", "tsmc_data", "intc_data", "nxpi_data", "avgo_data", "txn_data", "amd_data"]

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [153]:
soxx_data["Close"].corr(nvda_data["Close"])

0.9488188446068953

In [154]:
soxx_data["Close"].corr(tsmc_data["Close"])

-0.003757109863184324

In [155]:
soxx_data["Close"].corr(intc_data["Close"])

-0.5278427450864056

In [156]:
soxx_data["Close"].corr(nxpi_data["Close"])

0.8022136853111411

In [157]:
soxx_data["Close"].corr(avgo_data["Close"])

0.9453770724621768

In [201]:
pval = 1
single_pval = 1
for i1, ticker1 in enumerate(data_list):
    single_stats = adfuller(ticker1["Close"])
    if single_stats[1] < single_pval:
        single_pval = single_stats[1]
        single_min_ticker = data_list_names[i1]
        single_min = single_stats
    for i2, ticker2 in enumerate(data_list[i1+1:]):
        model = sm.OLS(ticker1["Close"], ticker2["Close"]).fit()
        test_res = adfuller(model.resid)
        if test_res[1] < pval:
            pval = test_res[1]
            pair_tickers = data_list_names[i1] + " " + data_list_names[i1+i2+1]
            pair = test_res
print(single_min_ticker)
print(single_min)
print(" ")
print(pair_tickers)
print(pair)
            
    

txn_data
(-2.985146098796241, 0.03630504539965115, 1, 229, {'1%': -3.4592326027153493, '5%': -2.8742454699025872, '10%': -2.5735414688888465}, 1090.7400906538635)
 
nxpi_data txn_data
(-2.738571606609665, 0.06759819572370056, 5, 225, {'1%': -3.4597521044060353, '5%': -2.874472927517147, '10%': -2.5736628197530864}, 1069.3197966495354)
